In [101]:
import ee
import geemap as emap

In [110]:
ee.Initialize()
ee.Authenticate()

Enter verification code: 4/1AX4XfWgZhcx17WTX0yUzvFYeJDyl4RbSoAl8z6Sn3GvA5NwH_QxpsV4_EoQ

Successfully saved authorization token.


In [111]:
Map=emap.Map()

**Append new element to EE List**

Please use `mlist`=mlist.add() because without mlist=, it does not work.

In [112]:
mlist=ee.List([])

for i in range(1,10):
    mlist=mlist.add(i)
mlist.getInfo()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

# <font color="blue"> 1. Preprocess Landsat 7 with error <end>

- **Landsat 7**

In [113]:
# Study area
site=ee.FeatureCollection("users/miketu72/Test_Area")
# Filter Landsat 7
ls7=ee.ImageCollection('LANDSAT/LE07/C01/T1_SR').filter(ee.Filter.date("2019-01-01","2020-12-31")).\
filter(ee.Filter.bounds(site)).filter(ee.Filter.lt("CLOUD_COVER",30))

In [114]:
# See the number of images over the period
ls7.size().getInfo()

6

In [115]:
# Visualize the first image
vis={"bands":["B3","B2","B1"],"min":-126.21, "max":1631.78}
Map.centerObject(site)
Map.addLayer(ls7.first(),vis,"LS7")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

- **Using focal_mean to fill empty values**

This function is based on `http://www.acgeospatial.co.uk/` website.

In [108]:
# Write the function using focal_mean
def focalMean(img):
    fillValue=img.focal_mean(2,"square","pixels",1)
    image=fillValue.blend(img)
    return image
# Map over the collection

ls7Coll=ls7.map(focalMean)

In [109]:
Map.addLayer(ls7Coll.first(),vis,"LS7 With Focal Mean")
Map # Not completely resolve the problem

Map(center=[21.24769080667753, 105.49588904011833], controls=(WidgetControl(options=['position', 'transparent_…

# <font color="blue"> 2. Select the greenest days of year in Landsat 8 <end>

- **Landsat 8**

In [88]:
# VN Area
VN=ee.FeatureCollection("users/miketu72/VN_Map")
ls8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filter(ee.Filter.date("2019-01-01","2020-12-31")).\
filter(ee.Filter.bounds(VN)).filter(ee.Filter.lt("CLOUD_COVER",60))

In [89]:
vis={"bands":["B4","B3","B2"],"min":-1415.85, "max":5000.87}
Map.addLayer(ls8.first(),vis,"LS8")
Map # Not completely resolve the problem

Map(center=[21.24769080667753, 105.49588904011833], controls=(WidgetControl(options=['position', 'transparent_…

- **Write a function to create a band of DOY, MOY and NDVI for each image**

In [90]:
def addImage(img):
    dayString=img.date().format("D")
    monthString=img.date().format("M")
    dayNum=ee.Number.parse(dayString)
    monthNum=ee.Number.parse(monthString)
    dayBand=img.select("B2").multiply(0).add(dayNum).rename("DOY").toInt() # the day of the year and needs to add .toInt()
    monthBand=img.select("B2").multiply(0).add(monthNum).rename("MOY").toInt() # Month of the year
    ndvi=img.expression("(NIR-RED)/(NIR+RED)",{
        'NIR':img.select("B5"),
        'RED':img.select("B4")}).rename("NDVI")
    image=img.addBands(dayBand).addBands(monthBand).addBands(ndvi)
    return image
collection=ls8.map(addImage)

In [91]:
# get the greenest pixel 
green=collection.qualityMosaic("NDVI")

In [92]:
vis_params = {'palette': ['fde725', ' c2df23', ' 86d549', ' 52c569', ' 2ab07f', ' 1e9b8a', ' 25858e', ' 2d708e', ' 38588c', ' 433e85', ' 482173', ' 440154'], 'min': 1.0, 'max': 365.0}
Map.addLayer(green.select("DOY").clip(VN),vis_params,"DOY")
Map

Map(center=[21.24769080667753, 105.49588904011833], controls=(WidgetControl(options=['position', 'transparent_…